In [1]:
import mysql.connector
import pandas as pd
import numpy as np

In [2]:
from mysql.connector import connect
from getpass import getpass

In [3]:
#Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:

db = connect(
host = 'localhost',
user = 'root',
passwd = getpass('Enter password:'),
database = 'sakila')

Enter password:········


In [ ]:
#Check the number of rentals for each customer for May

In [4]:
cursor = db.cursor()
query = 'select customer_id, count(rental_id) as number_of_rentals from rentals_may group by customer_id;'
cursor.execute(query)

In [5]:
df_may = pd.DataFrame(cursor.fetchall())
df_may

,0,1
0,130,2
1,459,1
2,408,3
3,333,1
4,222,5
...,...,...
515,191,2
516,351,1
517,10,1
518,136,1


In [ ]:
#Check the number of rentals for each customer for June

In [6]:
query = 'select customer_id, count(rental_id) as number_of_rentals from rentals_june group by customer_id;'
cursor.execute(query)

In [7]:
df_june = pd.DataFrame(cursor.fetchall())
df_june

,0,1
0,416,5
1,516,6
2,239,5
3,285,3
4,310,6
...,...,...
585,412,1
586,335,1
587,226,2
588,22,1


In [ ]:
#Write a function that checks if customer borrowed more or less films in the month of June as compared to May.
#Hint: For this part, you can create a join between the two dataframes created before, using the merge function available for pandas dataframes. 

In [9]:
def compare1(a,b):
    x = a.merge(b, how='outer', on=0, suffixes=('rentals_in_may', 'rentals_in_june')) # merge dataframes with may and june data on customer_id(0)
    x = x.fillna(0) # fill NaNs to calculate
    
    conditions = [x.iloc[:,1] > x.iloc[:,2],  # 1. If nº borrowed films in may > than nº of borrowed films in june
    x.iloc[:,1] < x.iloc[:,2]]                # 2. If nº borrowed films in june > than nº of borrowed films in may
    
    choices = ['May', 'June']                 # 'May' if condition 1. is met, otherwise 'June'
    
    x['month_with_more_rentals'] = np.select(conditions, choices, default='Tie') 
    return x


In [11]:
df_compare = compare1(df_may,df_june)

In [12]:
df_compare

,0,1rentals_in_may,1rentals_in_june,month_with_more_rentals
0,130,2.0,6.0,June
1,459,1.0,7.0,June
2,408,3.0,3.0,Tie
3,333,1.0,4.0,June
4,222,5.0,2.0,May
...,...,...,...,...
593,598,0.0,1.0,June
594,487,0.0,1.0,June
595,555,0.0,1.0,June
596,335,0.0,1.0,June


In [13]:
def compare2(a,b):
    x = a.merge(b, how='outer', on=0, suffixes=('rentals_in_may', 'rentals_in_june')) 
    x = x.fillna(0) # fill NaNs to calculate
    
    if x.iloc[:,1].sum() > x.iloc[:,2].sum(): 
        return 'May'                          
    else:
        return 'June'                         

In [16]:
print ('Month with more rentals:', compare2(df_may,df_june))

Month with more rentals: June
